<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Web/blob/main/141-Llamaindex01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## LlamaIndex <> Langchain Integrations

이 데모 노트북은 LlamaIndex와 Langchain 간의 통합을 제공할 수 있는 방법을 보여줍니다. 다음 예제를 제공합니다.:
- Langchain 에이전트와 함께 호출 가능한 도구로 LlamaIndex 사용
- LlamaIndex를 메모리 모듈로 사용 이를 통해 Langchain 챗봇에 임의의 양의 대화 기록을 삽입할 수 있습니다!

In [1]:
!pip install langchain langchain-core langchain-community
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.86.0
    Uninstalling openai-1.86.0:
      Successfully uninstalled openai-1.86.0


In [2]:
!pip install llama-index pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 122.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.8/266.8 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 734.3/734.3 kB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 12.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0


In [3]:
from google.colab import userdata
import openai
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
openai.api_key  = os.environ["OPENAI_API_KEY"]

In [4]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

#### Using LlamaIndex as a Callable Tool

In [5]:
from langchain.agents import Tool
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

In [6]:
!mkdir data

In [ ]:
import urllib.request

url = 'https://github.com/JSJeong-me/GPT-Web/raw/main/images/paul_graham_essay.txt'
filename = './data/paul_graham_essay.txt'

urllib.request.urlretrieve(url, filename)

# 파일 내용 확인 (처음 10줄 출력)
with open(filename, 'r', encoding='utf-8') as f:
    for _ in range(10):
        print(f.readline().rstrip())


In [8]:
documents = SimpleDirectoryReader("./data").load_data()

In [ ]:
documents

In [12]:
len(documents)

1

In [9]:
index = VectorStoreIndex.from_documents(documents=documents)

In [14]:
tools = [
    Tool(
        name="LlamaIndex",
        func=lambda q: str(index.as_query_engine().query(q)),
        description="useful for when you want to answer questions about the author. The input to this tool should be a complete english sentence.",
        return_direct=True,
    ),
]

In [15]:
# set Logging to DEBUG for more detailed outputs
memory = ConversationBufferMemory(memory_key="chat_history")

<ipython-input-15-4115651345>:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history")


In [22]:
llm = ChatOpenAI(model= 'gpt-4.1-2025-04-14', temperature=0)  # model='',

In [23]:
agent_executor = initialize_agent(
    tools, llm, agent="conversational-react-description", memory=memory
)

In [ ]:
agent_executor.invoke("LlamaIndex로 읽어 드린 문장의 내용을 5줄로 요약해 주세요")

## "LlamaIndex로 읽어 든 문장은 저자가 Lisp에 대한 책을 쓰기 시작하면서 Lisp에 대한 지식을 향상시키고자 했으며, 컴퓨터 과학 분야에서 이론과 시스템의 조화로운 연합을 언급하며, 시스템 작업의 한계와 불만을 표현하고, 예전의 컴퓨터 기술 회사들이 Moore's Law에 밀려가며 망하는 상황을 설명하며, 마지막으로 Lisp에 대한 글을 쓴 후에 받은 비판에 대한 경험을 언급하고 있습니다."

In [ ]:
agent_executor.invoke("이글의 주인공 이름은?")

In [ ]:
agent_executor.invoke("이글의 주인공의 대학 입학전 관한 내용 알려 주세요")

In [ ]:
agent_executor.run(input="hi, i am bob")

In [ ]:
agent_executor.run(input="what's my name?")

In [ ]:
agent_executor.run(input="who is the author of this document?")

In [ ]:
agent_executor.run(input="please summarize the document")

In [ ]:
agent_executor.run(input="please summarize the document in Korean")